# Memory savings using categoricals

In [2]:
#!/usr/bin/env python
import pandas as pd

## Without categoricals

In [14]:
# columns:
# Name,Position Title,Department,Employee Annual Salary
df_no_cat = pd.read_csv(
    "../DATA/city-of-chicago-salaries.csv",
    converters={'Employee Annual Salary':lambda s: s.strip('$')},
)
df_no_cat.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32054 entries, 0 to 32053
Data columns (total 4 columns):
Name                      32054 non-null object
Position Title            32054 non-null object
Department                32054 non-null object
Employee Annual Salary    32054 non-null object
dtypes: object(4)
memory usage: 8.5 MB


In [9]:
print(df_no_cat.describe())

                       Name  Position Title Department Employee Annual Salary
count                 32054           32054      32054                  32054
unique                31795            1098         35                   1097
top     RODRIGUEZ,  RICHARD  POLICE OFFICER     POLICE               78012.00
freq                      4            9432      13623                   2750


## With categoricals

In [11]:
df_cat = pd.read_csv(
    "../DATA/city-of-chicago-salaries.csv",
    converters={'Employee Annual Salary':lambda s: float(s[1:])},
    dtype={"Position Title": "category", "Department": "category"},
)


,Name,Position Title,Department,Employee Annual Salary
0,"AARON, ELVIA J",WATER RATE TAKER,WATER MGMNT,85512.0
1,"AARON, JEFFERY M",POLICE OFFICER,POLICE,75372.0
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,80916.0
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,99648.0
4,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,89440.0


In [64]:
print(df_cat.describe())

       Employee Annual Salary
count            32054.000000
mean             75070.385710
std              23180.297928
min                  0.960000
25%              69576.000000
50%              78012.000000
75%              87303.000000
max             260004.000000


In [13]:
df_cat.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32054 entries, 0 to 32053
Data columns (total 4 columns):
Name                      32054 non-null object
Position Title            32054 non-null category
Department                32054 non-null category
Employee Annual Salary    32054 non-null float64
dtypes: category(2), float64(1), object(1)
memory usage: 2.7 MB


## With categoricals (and splitting name)

In [22]:
df_cat_split = pd.read_csv(
    "../DATA/city-of-chicago-salaries.csv",    
    dtype={"Position Title": "category", "Department": "category"},
    converters={'Employee Annual Salary':lambda s: float(s[1:])},
)
df_cat_split['Last Name'], df_cat_split['First Name'] = df_cat_split.Name.str.split(',', 1).str
df_cat_split.drop('Name', axis=1, inplace=True)
df_cat_split['Last Name'] = df_cat_split['Last Name'].astype('category')
df_cat_split['First Name'] = df_cat_split['First Name'].astype('category')

df_cat_split.head()

,Position Title,Department,Employee Annual Salary,Last Name,First Name
0,WATER RATE TAKER,WATER MGMNT,85512.0,AARON,ELVIA J
1,POLICE OFFICER,POLICE,75372.0,AARON,JEFFERY M
2,CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,80916.0,AARON,KIMBERLEI R
3,CIVIL ENGINEER IV,WATER MGMNT,99648.0,ABAD JR,VICENTE M
4,ELECTRICAL MECHANIC,AVIATION,89440.0,ABBATACOLA,ROBERT J


In [26]:
print(df_cat_split["First Name"].describe())

count           32054
unique          11644
top         MICHAEL J
freq              280
Name: First Name, dtype: object


## Results
Let's look at memory usage for all 3 dataframes

In [17]:

df_no_cat.memory_usage(deep=True)


Index                          80
Name                      2378643
Position Title            2407193
Department                2074713
Employee Annual Salary    2085644
dtype: int64

In [71]:
print(df_cat.memory_usage(deep=True))
print()

Index                          80
Name                      2378643
Position Title             194406
Department                  35727
Employee Annual Salary     256432
dtype: int64



In [72]:
print(df_cat_split.memory_usage(deep=True))
print()

Index                          80
Position Title             194406
Department                  35727
Employee Annual Salary     256432
First Name                1606450
Last Name                 1167801
dtype: int64



In [75]:
print(df["Department"].value_counts())

POLICE                   13623
FIRE                      4731
STREETS & SAN             2070
WATER MGMNT               1857
OEMC                      1292
AVIATION                  1218
TRANSPORTN                1168
PUBLIC LIBRARY             926
GENERAL SERVICES           913
FAMILY & SUPPORT           727
HEALTH                     573
FINANCE                    520
LAW                        450
CITY COUNCIL               378
BUILDINGS                  242
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           181
BOARD OF ELECTION          110
DoIT                        99
MAYOR'S OFFICE              99
CITY CLERK                  86
IPRA                        85
CULTURAL AFFAIRS            79
PROCUREMENT                 71
HUMAN RESOURCES             68
ANIMAL CONTRL               61
INSPECTOR GEN               50
BUDGET & MGMT               44
ADMIN HEARNG                42
DISABILITIES                29
TREASURER                   25
HUMAN RELATIONS             18
BOARD OF

In [53]:
df_cat[
    (df_cat['Department'] == 'POLICE') &
    (df_cat['Position Title'].str.contains('DETECTIVE'))
].nlargest(10, 'Employee Annual Salary')


,Name,Position Title,Department,Employee Annual Salary
9427,"GALLAGHER, GEORGE J",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,96444.0
11713,"HART, TERENCE C",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,96444.0
15854,"LAS COLA, JAMES C",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,96444.0
24033,"RILEY, MARY P",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,96444.0
24769,"ROMIC, JEAN MARIE",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,96444.0
185,"AGOSTA, JOSEPH J",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,93192.0
234,"AGUIRRE, JOSEPH R",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,93192.0
707,"ANDERSON, JAMES E",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,93192.0
1327,"BALDWIN, JOHN R",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,93192.0
1545,"BARRERA, JOSEPH A",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,93192.0


In [57]:
df_cat[
    (df_cat['Department'] == 'POLICE') &
    (df_cat['Position Title'].str.contains('DETECTIVE'))
].idxmax(1, 'Employee Annual Salary')


ValueError: No axis named Employee Annual Salary for object type <class 'pandas.core.frame.DataFrame'>